<a href="https://colab.research.google.com/github/neerajthandayan/CourseProject/blob/main/Notebooks/Hate_Crime_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries

import pandas as pd
import numpy as np

In [ ]:
# Data Retrieved From: https://www.gov.uk/government/statistics/hate-crimes-england-and-wales-2011-to-2012--2

!wget 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/173000/hatecrimes1112tabs.xls'

--2021-04-19 18:09:52--  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/173000/hatecrimes1112tabs.xls
Resolving assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)... 151.101.0.144, 151.101.64.144, 151.101.128.144, ...
Connecting to assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)|151.101.0.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41984 (41K) [application/vnd.ms-excel]
Saving to: ‘hatecrimes1112tabs.xls.3’

hatecrimes1112tabs. 100%[===================>]  41.00K  --.-KB/s    in 0.01s   

2021-04-19 18:09:53 (3.34 MB/s) - ‘hatecrimes1112tabs.xls.3’ saved [41984/41984]



In [ ]:
hc = pd.read_excel('hatecrimes1112tabs.xls',sheet_name='HC.01', header=4)
hc = hc.iloc[:53,:3]
hc.dropna(inplace=True)
hc.rename(columns={'Unnamed: 0':'Geography','Race4':'Race'}, inplace=True)
hc.reset_index(inplace=True, drop=True)

In [ ]:
for i in range(len(hc)):
  hc['Geography'][i] = hc['Geography'][i].replace('5','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
hc['Ethnic_HC'] = hc.iloc[:,1:].sum(axis=1)
hc = hc[['Geography','Ethnic_HC']]
hc.head()

,Geography,Ethnic_HC
0,Cleveland,308.0
1,Durham,250.0
2,Northumbria,570.0
3,Cheshire,290.0
4,Cumbria,211.0


#### Total Offences Data

In [ ]:
# Total Offence Data Retrieved From: https://www.gov.uk/government/statistics/crimes-detected-in-england-and-wales-2011-to-2012

!wget 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/116437/hosb0812-pfatabs.xls'

--2021-04-19 18:09:54--  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/116437/hosb0812-pfatabs.xls
Resolving assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)... 151.101.0.144, 151.101.64.144, 151.101.128.144, ...
Connecting to assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)|151.101.0.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98304 (96K) [application/vnd.ms-excel]
Saving to: ‘hosb0812-pfatabs.xls.1’

hosb0812-pfatabs.xl 100%[===================>]  96.00K  --.-KB/s    in 0.02s   

2021-04-19 18:09:54 (5.29 MB/s) - ‘hosb0812-pfatabs.xls.1’ saved [98304/98304]



In [ ]:
off = pd.read_excel('hosb0812-pfatabs.xls', sheet_name='Table 1', header=6)
off = off.iloc[:63,:2].dropna().reset_index(drop=True)
off.columns = ['Geography','Total_Offences']
off.head()

,Geography,Total_Offences
0,Cleveland,43535.0
1,Durham,35331.0
2,Northumbria,75396.0
3,North East Region,154262.0
4,Cheshire,59829.0


#### Merging DataFrames to estimate Ethnic Hate Crime Rate

In [ ]:
ecr = hc.merge(off, on='Geography')
ecr['EHC_Rate'] = ecr['Ethnic_HC']/ecr['Total_Offences']
ecr.sort_values(by='EHC_Rate', ascending=False).head(10)

,Geography,Ethnic_HC,Total_Offences,EHC_Rate
5,Greater Manchester,3154.0,207523.0,0.015198
13,Leicestershire,957.0,68242.0,0.014024
20,West Midlands,2583.0,193532.0,0.013347
7,Merseyside,1137.0,96561.0,0.011775
34,Avon and Somerset,1289.0,115186.0,0.011191
28,Metropolitan Police,8590.0,814625.0,0.010545
24,Hertfordshire,653.0,62251.0,0.010490
11,West Yorkshire,1885.0,184085.0,0.010240
17,Staffordshire,652.0,65680.0,0.009927
27,"London, City of",59.0,6119.0,0.009642


In [ ]:
ecr[['Geography','EHC_Rate']].to_csv('ethnichatecrime_rate.csv')